In [1]:
# -*- coding: utf-8 -*-
import sqlite3
import pandas as pd
import csv

Note : rating[happy,boost,chill,loved,adventure,free]

In [2]:
conn = sqlite3.connect("Urban Data Hackathon/UTSEUS-shanghai-weibo-2012.db")
c = conn.cursor()

In [3]:
req = "SELECT ws.weibo_location_id, status\
      FROM weibo_status AS ws, weibo_location AS wl\
      WHERE wl.weibo_location_id = ws.weibo_location_id"

c.execute(req)
raw_data = c.fetchall()

In [4]:
labels = ['loc_id','status']

df = pd.DataFrame.from_records(raw_data,columns=labels)
df.shape[0]
emoji_status = df[df.status.str.contains("\[")]
#emoji_status

In [5]:
import array
# this function return all the emojis present in a status and the number of occurence
def emoji_list(status):
    status_emoji = {}
    emoji = ''
    flag = False
    for ch in status:
        if ch == ']':
            flag = False
            if emoji in status_emoji.keys():
                status_emoji[emoji] += 1
            else:
                status_emoji[emoji] = 1
            emoji=""
        else:
            if not flag:
                if ch == '[':
                    flag = True
            else:
                emoji += ch
    return status_emoji
test = emoji_list('东方明珠比十年前还没意思[右哼哼][左哼哼][懒得理你][兔子] 我在这里:http://t.cn/zl61aYv')
test


{'右哼哼': 1, '左哼哼': 1, '懒得理你': 1, '兔子': 1}

In [6]:
# this function return an array with 6 integers, giving the 6 scores for a given status
from operator import itemgetter
filename = 'emoji_status_rated.csv'



def status_rating(status):
    csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
    rating = [0] * 6
    status_compo = emoji_list(status)
    for key,value in status_compo.items():
        for row in csv_file:
            #print("On essaye ",key, row[1])
            if key == row[1]:
                rating[0] += float(row[4])*value # Happy
                rating[1] += float(row[5])*value # Boost
                rating[2] += float(row[6])*value # Chill
                rating[3] += float(row[7])*value # loved
                rating[4] += float(row[8])*value # Adventure
                rating[5] += float(row[9])*value # Freedom
                break       
    return rating

test2 = status_rating('The food just so so here [围脖] 我在#首尔焖鲜汇# http://t.cn/zjtAtq9')
test2

[3.0, 2.0, 1.0, 3.0, -2.0, 3.0]

In [7]:
# we here have all the weibo status with emojis. What we do here, using the previous function, is for each LOCATION
# we add all the ratings of the concerned status in the shape : {location_id, (a,b,c,d,e,f,g)}
import numpy as np
feeling_rating = {}

for row in emoji_status.iterrows():
    status = str(row[1].status)
    rate = status_rating(status) #rate will be like (a,b,c,d,e,f,g)  
    loc_id = row[1].loc_id
    ####    
    if loc_id in feeling_rating.keys():
        new_data = [0] * 6
        new_data[0] = feeling_rating[loc_id][0] + rate[0]
        new_data[1] = feeling_rating[loc_id][1] + rate[1]
        new_data[2] = feeling_rating[loc_id][2] + rate[2]
        new_data[3] = feeling_rating[loc_id][3] + rate[3]
        new_data[4] = feeling_rating[loc_id][4] + rate[4]
        new_data[5] = feeling_rating[loc_id][5] + rate[5]
        feeling_rating[loc_id] = new_data
    else:
        feeling_rating[loc_id] = rate


    

In [8]:
import pickle 
with open('svg_feeling_rating','wb') as handle:
    pickle.dump(feeling_rating,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
df_test = pd.DataFrame.from_dict(feeling_rating,orient='index')
df_test.to_csv('loc_feeling_rating.csv')

Normalize CSV

In [47]:
filename = 'loc_feeling_rating.csv'
csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv(filename)
df.describe()

,Unnamed: 0,0,1,2,3,4,5
count,39307.000000,39307.000000,39307.000000,39307.000000,39307.000000,39307.000000,39307.000000
mean,27165.698654,12.160684,13.234004,-0.282291,8.933014,13.077569,12.778462
std,16174.770981,64.435527,63.465762,26.338007,42.708594,40.130928,48.012542
min,8.000000,-2374.000000,-2154.000000,-1467.000000,-1097.000000,-220.000000,-850.000000
25%,12662.000000,-2.000000,-2.000000,-3.000000,0.000000,1.000000,0.000000
50%,27707.000000,4.000000,4.000000,1.000000,2.000000,4.000000,4.000000
75%,40875.500000,15.000000,15.000000,5.000000,10.000000,12.000000,12.000000
max,54678.000000,3381.000000,3607.000000,849.000000,2668.000000,1687.000000,2514.000000


In [42]:
filename = 'loc_feeling_rating.csv'
csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
max_happy = max(float(column[1].replace(',', '')) for column in csv_file)

csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
max_boost = max(float(column[2].replace(',', '')) for column in csv_file)

csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
max_chill = max(float(column[3].replace(',', '')) for column in csv_file)

csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
max_loved = max(float(column[4].replace(',', '')) for column in csv_file)

csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
max_adventure = max(float(column[5].replace(',', '')) for column in csv_file)

csv_file = csv.reader(open(filename, 'r',encoding='UTF-8'))
max_free = max(float(column[6].replace(',', '')) for column in csv_file)

In [43]:
print(max_happy)
print(max_boost)
print(max_chill)
print(max_loved)
print(max_adventure)
print(max_free)

3381.0
3607.0
849.0
2668.0
1687.0
2514.0
